In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import time
import copy
import string
import json
import pickle as pk

from collections import Counter

from math import sin, asin, cos, radians, fabs, sqrt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

In [11]:
############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

In [12]:
##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()

In [13]:
####################   proba()  把 staying list  转换为概率特征    ############
#  输入: staying list   一个POI上的停留时间list ;  把停留时间化为了9个时段0::8
#  输出: st_f      停留时间概率分布，  
def proba(st):
    t = st.copy()
    for i in range(0, len(st)):
    
        if 10 <= st[i] < 30:
            t[i] = 0
        elif 30 <= st[i] < 60:
            t[i] = 1
        elif 60 <= st[i] < 90:
            t[i] = 2
        elif 90 <= st[i] < 120:
            t[i] = 3
        elif 120 <= st[i] < 150:
            t[i] = 4
        elif 150 <= st[i] < 180:
            t[i] = 5
        elif 180 <= st[i] < 210:
            t[i] = 6
        elif 210 <= st[i] < 240:
            t[i] = 7
        else:
            t[i] = 8
    
    b = len(t)
    a = Counter(t)
    #print(a)
    stf = [0]*9        # staying list 转化为 fecture vector
    #print(st_f)

    for key, value in a.items():
        stf[key] = round(value/b, 3)
        
    return stf   

In [15]:
########################  读取 st_poi_tree ########################
filename = ".\\os_file\\st_poi_tree.pkl"
st_poi_tree = read_q(filename)
len(st_poi_tree["enter"]["bar"])

27

In [16]:
a4 = []
b4 = 0
for item1 in st_poi_tree:
    #print(item1)
    for item2 in st_poi_tree[item1]:
        b4 += 1
        print(item2)
        a4.append(len(st_poi_tree[item1][item2]))
        
a4

bar
bath
bbulid
cinema
farm
ktv
scenic
tea
buffet
chfood
ffood
hotpot
jkcui
sweet
wfood
ehotel
fivestar
fourstar
raccom
threestar
kgartern
mschool
pschool
train
univer
bookstore
fmarket
mall
smarket


[27,
 124,
 356,
 2,
 7,
 38,
 515,
 8,
 12,
 68,
 374,
 88,
 24,
 80,
 32,
 411,
 0,
 0,
 0,
 0,
 0,
 57,
 71,
 568,
 112,
 38,
 20,
 8,
 594]

In [9]:
st_poi_tree["enter"]["cinema"]

[{'id': '枫花园汽车电影院',
  'location': {'lat': 39.9643, 'lng': 116.486526},
  'staying': [21.96666662581265],
  'category': 'cinema'},
 {'id': '北京UME国际影城(华星店)',
  'location': {'lat': 39.973648, 'lng': 116.332704},
  'staying': [14.000000118976459, 14.000000118976459],
  'category': 'cinema'}]

In [29]:
######################   KMeans  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic
def k_means(list_dic, n):
    

    print("调用 K-means，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = KMeans(n_clusters = n ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d

In [30]:
#####################  第一次k-means   ###############################
# n    聚类个数
filename = ".\\os_file\\st_poi_kmeans1.pkl"
n = 4
list_dic = []
st_poi_kmeans1 = copy.deepcopy(st_poi_tree)
a = 0
for item1 in st_poi_tree:
    print("第一大类 ", item1)
    b = 0
    
    for item2 in st_poi_tree[item1]:
       
        list_dic = st_poi_tree[item1][item2]
        print("    当前小类： ", item2, "POI长度为： ",len(list_dic))
        b += len(list_dic)
        
        if len(list_dic) < 10:
            continue
        
        k_m = k_means(list_dic, n)    
       
        st_poi_kmeans1[item1][item2] = k_m
    print("第一大类： ",item1,"共有POI：",b, "个")
    
    a += b    
print("共处理POI： ",a," 个")
write_q(st_poi_kmeans1, filename)

第一大类  enter
    当前小类：  bar POI长度为：  27
调用 K-means，聚类前list的长度为：  27
参与聚类的list 的长度 27
聚类后，第一类list的长度为：  18
聚类后，第一类list的长度为：  6
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  2
聚类后，两层列表总长度为：  27
    当前小类：  bath POI长度为：  124
调用 K-means，聚类前list的长度为：  124
参与聚类的list 的长度 124
聚类后，第一类list的长度为：  12
聚类后，第一类list的长度为：  70
聚类后，第一类list的长度为：  9
聚类后，第一类list的长度为：  33
聚类后，两层列表总长度为：  124
    当前小类：  bbulid POI长度为：  356
调用 K-means，聚类前list的长度为：  356
参与聚类的list 的长度 356
聚类后，第一类list的长度为：  211
聚类后，第一类list的长度为：  33
聚类后，第一类list的长度为：  95
聚类后，第一类list的长度为：  17
聚类后，两层列表总长度为：  356
    当前小类：  cinema POI长度为：  2
    当前小类：  farm POI长度为：  7
    当前小类：  ktv POI长度为：  38
调用 K-means，聚类前list的长度为：  38
参与聚类的list 的长度 38
聚类后，第一类list的长度为：  28
聚类后，第一类list的长度为：  4
聚类后，第一类list的长度为：  5
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  38
    当前小类：  scenic POI长度为：  515
调用 K-means，聚类前list的长度为：  515
参与聚类的list 的长度 515
聚类后，第一类list的长度为：  62
聚类后，第一类list的长度为：  255
聚类后，第一类list的长度为：  149
聚类后，第一类list的长度为：  49
聚类后，两层列表总长度为：  515
    当前小类：  tea POI长度为：  8
第一大类：  enter 共有POI： 1

In [1]:
#############################  2次  KMeans   ######################
#  输入 st_poi_kmeans1
#  输出 st_poi_kmeans2
filename = ".\\os_file\\st_poi_kmeans2.pkl"
n = 4
#list_dict = []
st_poi_kmeans2 = copy.deepcopy(st_poi_kmeans1)
a = 0
for item1 in st_poi_kmeans1:
    print("第一大类： ",item1)
    
    a1= 0
    for item2 in st_poi_kmeans1[item1]:
        b = 0
        st = st_poi_kmeans1[item1][item2]
        print("    当前小类： ",item2)
        if 0 <= len(st)< 4 or 4 < len(st) < 10:  #未动过的点
            b += len(st)
            a1 += b
            continue
        
        if type(st[0]).__name__ == 'dict':    # len =4 的未动过的点    判断对象类型
            b += len(st)
            a1 += b
            continue
        
        #type(st[0]).__name__ == 'list':
        
        for i in range(len(st)):    
            list_dict = st[i]
            b += len(list_dict)
            
            if len(list_dict) < 10:
                continue
            
            k_m = k_means(list_dict, n)
            
            st_poi_kmeans2[item1][item2][i] = k_m
        
        a1 += b
        print(item2,"共处理POI： ",b,"个")
    a += a1
    print(item1,"共处理POI： ",a1,"个")
print("第二次K-means 共处理POI：",a,"个")
            
write_q(st_poi_kmeans2,filename)

NameError: name 'copy' is not defined

In [92]:
st_poi_kmeans2["enter"]["bar"][1]

[{'id': '半山酒吧',
  'location': {'lat': 39.973151, 'lng': 116.48791299999999},
  'staying': [35.41666665812954, 151.58333333325572],
  'category': 'bar'},
 {'id': 'juice bar',
  'location': {'lat': 39.918248, 'lng': 116.31093600000001},
  'staying': [42.083333290647715, 151.28333336790092],
  'category': 'bar'},
 {'id': '猫头鹰音乐部落(知春路店)',
  'location': {'lat': 39.980621, 'lng': 116.34581899999999},
  'staying': [155.9666666854173, 184.26666672108695],
  'category': 'bar'},
 {'id': '钼molybdenum酒吧',
  'location': {'lat': 39.983653000000004, 'lng': 116.489019},
  'staying': [15.200000064214692, 32.333333326969296],
  'category': 'bar'},
 {'id': 'hoper',
  'location': {'lat': 39.950175, 'lng': 116.408971},
  'staying': [30.649999977322295,
   93.5166666295845,
   68.55000004637986,
   40.466666591819376],
  'category': 'bar'},
 {'id': '星巴克(北京海润店)',
  'location': {'lat': 39.982292, 'lng': 116.48978999999999},
  'staying': [52.166666738921776],
  'category': 'bar'}]

In [86]:
a4 = []
b4 = 0
for item1 in st_poi_kmeans2:
    #print(item1)
    for item2 in st_poi_kmeans2[item1]:
        b4 += 1
        #print(item2)
        a4.append(len(st_poi_kmeans2[item1][item2]))
        
a4

[4,
 4,
 4,
 2,
 7,
 4,
 4,
 8,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 8,
 4]